In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [9]:
def asignarPV(name, dic):
    return dic[name]

In [10]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [11]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [12]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [13]:
p = p[['property_type','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos', 'place_value']]

# Vamos a hacer lo mismo con greed search

In [14]:
busqP = p[['property_type','surface_total_in_m2','surface_covered_in_m2',\
         'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos', 'place_value']]

resP = p[['price_usd_per_m2']]

In [16]:
param = {'n_neighbors': [x for x in xrange(1,30)], 'p':[1,2]}
knn2 = neighbors.KNeighborsRegressor()

In [17]:
gs = GridSearchCV( estimator=knn2, param_grid=param )

In [18]:
gs.fit(busqP, resP )

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'p': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
score = gs.best_score_*100
par = gs.best_params_

In [25]:
par

{'n_neighbors': 27, 'p': 2}